In [1]:
from lib.db import MongoClient
from lib.helper import get_faults
import csv
import pymongo

In [2]:
c = MongoClient("mongodb://localhost:27017")

In [3]:
csv_file = "UK stations.csv"
mapping = dict()
with open(csv_file, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    counter = 0
    for row in spamreader:
        counter += 1
        if counter == 1:
            continue
        title = row[0]
        lat = row[2]
        long = row[3]
        mapping[title] = {"lat": lat, "long": long}

In [4]:
mapping

{'Abbey Wood': {'lat': '51.4910605627019', 'long': '0.121394025366647'},
 'Aber': {'lat': '51.5749691125956', 'long': '-3.22982905106563'},
 'Abercynon': {'lat': '51.6447124034837', 'long': '-3.32698855170195'},
 'Aberdare': {'lat': '51.7150618291708', 'long': '-3.44308409477462'},
 'Aberdeen': {'lat': '57.1437058328662', 'long': '-2.09868538622245'},
 'Aberdour': {'lat': '56.0545955695197', 'long': '-3.30056061487314'},
 'Aberdovey': {'lat': '52.5439615025967', 'long': '-4.05706796158074'},
 'Abererch': {'lat': '52.8985884658899', 'long': '-4.37417647351973'},
 'Abergavenny': {'lat': '51.8166930409601', 'long': '-3.00964730040687'},
 'Abergele and Pensarn': {'lat': '53.2945771647609',
  'long': '-3.58262268840784'},
 'Aberystwyth': {'lat': '52.4140498933205', 'long': '-4.08189442746786'},
 'Accrington': {'lat': '53.7529733751372', 'long': '-2.36954810226505'},
 'Achanalt': {'lat': '57.6096018732679', 'long': '-4.91385256332031'},
 'Achnasheen': {'lat': '57.5792959197415', 'long': '-5.

In [7]:
updates = []
count = 0
for station in c.get_all_stations():
    _id = station.get("_id")
    station = station.get("station_name")
    station = station.replace(" Station", "")
    station = station.replace("London ", "")
    station = station.replace(" HL", "")
    station = station.replace(" St", "")

    try:
        loc = mapping[station]
    except:
        continue
    lat = loc.get("lat")
    long = loc.get("long")
    count += 1
    updates.append(pymongo.UpdateOne({"_id": _id}, {"$set": {"lat": lat, "long": long}}))
if updates:
    c.stations.bulk_write(updates)
print(count)

30
